In [7]:
import re
import csv
from collections import Counter

# Constants
LOG_FILE = "E:/iot project 2/sample.log"
CSV_OUTPUT = "E:/iot project 2/log_analysis_results3.csv"
FAILED_LOGIN_THRESHOLD = 5

def parse_log_file(log_file):
    """Parses the log file and extracts log entries."""
    with open(log_file, 'r') as file:
        return file.readlines()

def count_requests_per_ip(log_entries):
    """Counts the number of requests made by each IP address."""
    ip_count = Counter()
    for entry in log_entries:
        match = re.match(r'(\d+\.\d+\.\d+\.\d+)', entry)
        if match:
            ip_count[match.group(1)] += 1
    return ip_count

def find_most_accessed_endpoint(log_entries):
    """Finds the most frequently accessed endpoint."""
    endpoint_count = Counter()
    for entry in log_entries:
        match = re.search(r'"[A-Z]+\s(\S+)\sHTTP/\d\.\d"', entry)
        if match:
            endpoint_count[match.group(1)] += 1
    if endpoint_count:
        return endpoint_count.most_common(1)[0]
    return None, 0

def detect_suspicious_activity(log_entries, threshold):
    """Detects IP addresses with suspicious failed login attempts."""
    failed_logins = Counter()
    for entry in log_entries:
        if '401' in entry or "Invalid credentials" in entry:
            match = re.match(r'(\d+\.\d+\.\d+\.\d+)', entry)
            if match:
                failed_logins[match.group(1)] += 1
    return {ip: count for ip, count in failed_logins.items() if count > threshold}

def save_results_to_csv(ip_requests, most_accessed, suspicious_activities, output_file):
    """Saves the analysis results to a CSV file."""
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # Requests per IP
        writer.writerow(["IP Address", "Request Count"])
        for ip, count in ip_requests.items():
            writer.writerow([ip, count])
        writer.writerow([])

        # Most Accessed Endpoint
        writer.writerow(["Most Accessed Endpoint", "Access Count"])
        writer.writerow(most_accessed)
        writer.writerow([])

        # Suspicious Activities
        writer.writerow(["IP Address", "Failed Login Attempts"])
        for ip, count in suspicious_activities.items():
            writer.writerow([ip, count])

def main():
    log_entries = parse_log_file(LOG_FILE)

    # Analysis
    ip_requests = count_requests_per_ip(log_entries)
    most_accessed = find_most_accessed_endpoint(log_entries)
    suspicious_activities = detect_suspicious_activity(log_entries, FAILED_LOGIN_THRESHOLD)

    # Display results
    print("Requests per IP:")
    for ip, count in ip_requests.items():
        print(f"{ip} - {count}")

    print("\nTop 5 Requests per IP:")
    for ip, count in ip_requests.most_common(5):
        print(f"{ip} - {count}")

    print("\nMost Frequently Accessed Endpoint:")
    if most_accessed:
        print(f"{most_accessed[0]} (Accessed {most_accessed[1]} times)")

    print("\nSuspicious Activity Detected:")
    for ip, count in suspicious_activities.items():
        print(f"{ip} - {count} failed login attempts")

    # Save results to CSV
    save_results_to_csv(ip_requests, most_accessed, suspicious_activities, CSV_OUTPUT)
    print(f"\nResults saved to {CSV_OUTPUT}")

if __name__ == "__main__":
    main()


Requests per IP:
192.168.1.1 - 7
203.0.113.5 - 8
10.0.0.2 - 6
198.51.100.23 - 8
192.168.1.100 - 5

Top 5 Requests per IP:
203.0.113.5 - 8
198.51.100.23 - 8
192.168.1.1 - 7
10.0.0.2 - 6
192.168.1.100 - 5

Most Frequently Accessed Endpoint:
/login (Accessed 13 times)

Suspicious Activity Detected:
203.0.113.5 - 8 failed login attempts

Results saved to E:/iot project 2/log_analysis_results3.csv
